# 30M Resistance/Support Detection

In [95]:
import pandas as pd
df = pd.read_csv("data/monthly_segments/CL_30m_2024-09.csv")
df.tail()

,datetime,open,high,low,close,volume
989,2024-09-30 21:30:00,68.23,68.29,68.20,68.23,369
990,2024-09-30 22:00:00,68.23,68.28,68.22,68.25,290
991,2024-09-30 22:30:00,68.25,68.28,68.19,68.25,798
992,2024-09-30 23:00:00,68.26,68.42,68.26,68.34,493
993,2024-09-30 23:30:00,68.35,68.35,68.26,68.28,462


In [96]:
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail()

,datetime,open,high,low,close,volume
989,2024-09-30 21:30:00,68.23,68.29,68.20,68.23,369
990,2024-09-30 22:00:00,68.23,68.28,68.22,68.25,290
991,2024-09-30 22:30:00,68.25,68.28,68.19,68.25,798
992,2024-09-30 23:00:00,68.26,68.42,68.26,68.34,493
993,2024-09-30 23:30:00,68.35,68.35,68.26,68.28,462


In [97]:
# Slice for plotting; adjust as needed
s = 0
e = 450

# Support and Resitance FUNCTIONS

In [98]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

#support(df,46,3,2)

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1
#resistance(df, 30, 3, 5)


In [99]:
dfpl = df[s:e]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color='green',
                decreasing_line_color='red'
                )])

fig.show()

In [100]:
sr = []
n1=4
n2=2
for row in range(n1, len(df) - n2): #len(df)-n2
    if support(df, row, n1, n2):
        sr.append((row,df.low[row],1))
    if resistance(df, row, n1, n2):
        sr.append((row,df.high[row],2))
print(sr)


[(4, np.float64(72.95), 1), (22, np.float64(73.91), 2), (37, np.float64(74.41), 2), (68, np.float64(72.06), 1), (69, np.float64(72.06), 1), (76, np.float64(70.38), 1), (86, np.float64(70.43), 2), (106, np.float64(70.11), 2), (109, np.float64(69.19), 1), (133, np.float64(68.82), 1), (134, np.float64(68.82), 1), (284, np.float64(69.08), 2), (323, np.float64(66.43), 2), (332, np.float64(65.91), 1), (358, np.float64(65.63), 1), (368, np.float64(67.05), 1), (390, np.float64(68.5), 2), (403, np.float64(67.81), 1), (406, np.float64(69.81), 2), (423, np.float64(69.24), 1), (425, np.float64(69.5), 2), (442, np.float64(69.92), 2), (459, np.float64(68.47), 1), (465, np.float64(69.39), 2), (471, np.float64(68.7), 1), (474, np.float64(69.0), 2), (475, np.float64(69.0), 2), (498, np.float64(70.7), 2), (530, np.float64(70.75), 2), (550, np.float64(71.92), 2), (560, np.float64(69.91), 1), (583, np.float64(68.58), 1), (594, np.float64(70.26), 2), (604, np.float64(68.95), 1), (648, np.float64(70.96), 1)

In [ ]:
dfpl = df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

start = dfpl.index[0]
end = dfpl.index[-1]
levels_in_slice = [x for x in sr if s <= x[0] < e]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

for row, yval, _ in levels_in_slice:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval
                  )

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=f'Support/Resistance Bands'
)

fig.add_annotation(
    x=1, y=1,
    xref='paper', yref='paper',
    text=f'Number of levels: {len(levels_in_slice)}',
    showarrow=False,
    xanchor="right", yanchor="top",
    font=dict(size=16, color='white')
)

fig.show()


In [103]:
plotlist2 = [x[1] for x in sr if x[2]==2]
plotlist1 = [x[1] for x in sr if x[2]==1]
initial_levels = len(sr)
collapse_thresh = 0.5
plotlist1.sort()
plotlist2.sort()

for i in range(1,len(plotlist1)):
    if(i>=len(plotlist1)):
        break
    if abs(plotlist1[i]-plotlist1[i-1]) <= collapse_thresh:
        plotlist1.pop(i)

for i in range(1,len(plotlist2)):
    if(i>=len(plotlist2)):
        break
    if abs(plotlist2[i]-plotlist2[i-1]) <= collapse_thresh:
        plotlist2.pop(i)

print(f"Collapsed {initial_levels-len(plotlist1)-len(plotlist2)} levels.")


Collapsed 25 levels.


In [ ]:
dfpl = df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

start = dfpl.index[0]
end = dfpl.index[-1]

fig = go.Figure(data=[go.Candlestick(
                x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color='green',
                decreasing_line_color='red'
                )],
                )

for yval in plotlist1:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval,
                  line=dict(color="Blue",width=2)
                  )

for yval in plotlist2:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval,
                  line=dict(color="Red",width=2)
                  )

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=f'Support/Resistance Bands'
)

fig.show()


In [105]:
rr = []
for row in range(n1, len(df) - n2): #len(df)-n2
    if support(df, row, n1, n2):
        ss.append((row,df.low[row]))
    if resistance(df, row, n1, n2):
        rr.append((row,df.high[row]))


In [ ]:
dfpl = df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

start = dfpl.index[0]
end = dfpl.index[-1]
ss_slice = [(r, y) for (r, y) in ss if s <= r < e]
rr_slice = [(r, y) for (r, y) in rr if s <= r < e]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color='green',
                decreasing_line_color='red')])

for row, yval in ss_slice:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval,
                  line=dict(color="#8afffd",width=2),
                  name='Support',
                  opacity=0.75
                  )

for row, yval in rr_slice:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval,
                  line=dict(color="#ff3c69",width=2),
                  name='Resistance',
                  opacity=0.75
                  )

fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=f'Support/Resistance Bands'
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Time-decayed opacity vs last candle within slice
dfpl = df.iloc[s:e]
df_idx = pd.to_datetime(dfpl["datetime"])
last_time = df_idx.max()
first_time = df_idx.min()
span_sec = max((last_time - first_time).total_seconds(), 1)

time_decay = 3.0

ss_slice = [(r, y) for (r, y) in ss if s <= r < e]
rr_slice = [(r, y) for (r, y) in rr if s <= r < e]


def time_decay_levels(levels):
    out = []
    for row, level in levels:
        level_time = df_idx.iloc[row - s] if 0 <= row - s < len(df_idx) else last_time
        age_sec = max((last_time - level_time).total_seconds(), 0)
        age_norm = age_sec / span_sec
        strength = float(np.exp(-time_decay * age_norm))
        out.append((row, level, strength))
    return out

ss_strengths = time_decay_levels(ss_slice)
rr_strengths = time_decay_levels(rr_slice)

fig = go.Figure(data=[go.Candlestick(
    x=df_idx,
    open=dfpl['open'], high=dfpl['high'], low=dfpl['low'], close=dfpl['close'],
    increasing_line_color='green', decreasing_line_color='red'
)])

x_start = df_idx.min()
x_end = df_idx.max()

for _, level, strength in ss_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=x_start, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(30,144,255)', width=2),
        opacity=strength,
    )

for _, level, strength in rr_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=x_start, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(220,20,60)', width=2),
        opacity=strength,
    )

fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title='Support/Resistance (Time Decay vs Last Candle)'
)

fig.show()
